При работе с гибкими инструментами стоит придерживаться распространенных подходов.
Это касается организации кода, работы с данными, написания моделей и т.д.

**План этого семинара:**

1. Работа с данными в pytorch. Dataset&Dataloader
2. Написание сверточной сети
3. Написание кода тренировки
4. Исследование затухания градиентов на примере полносвязных сетей.

## 1. Работа с данными в pytorch

https://pytorch.org/docs/stable/data.html
Обычно работа с данными декомпозирована на два класса:
    
### `torch.utils.data.Dataset`

Класс для работы с семплами. Сюда часто добавляют логику скачивания датасета, препроцессинг и аугментации.

Для работы со своими данными нужно отнаследоваться от этого класса и реализовать два метода: `__len__` и `__getitem__`.
Сначала мы воспользуемся готовым датасетом из [`torchvision.datasets`](https://pytorch.org/docs/stable/torchvision/datasets.html)

### `torch.utils.data.Dataloader`

Загрузчик данных, загружает семплы из Dataset, занимается семплирование, батчеванием, перемешиванием и т.д.
Умеет в multiprocessing, это необходимо при работе со сколько-нибудь большими датасетами.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

# папку для загрузки можно поменять
download_path = '/tmp'
mnist_train = datasets.MNIST(download_path, train=True, download=True, transform=transforms.ToTensor())
mnist_val = datasets.MNIST(download_path, train=False, download=True, transform=transforms.ToTensor())

**Задание 0. (0.1 балла)**
1. В каком виде возвращает семплы итератор по `mnist_train`?
2. Отобразите несколько примеров

In [ ]:
# напишите ответ текстом или кодом здесь
<your code here>

In [ ]:
# обязательно смотрите на то, в каком виде возвращаются семплы
plt.figure(figsize=[6, 6])
for i in range(4):
    plt.subplot(2, 2, i + 1)
    img, label = <your code here>
    
    plt.title("Label: {}".format(label))
    plt.imshow(img, cmap='gray')

## Сверточные сети

Мы рассмотрим сверточные сети на примере MNIST, заодно поучимся пользоваться стандартными pytorch-классами для работы с данными.

В случае картинок, обычно работают с входными тензорами размера `[batch_size, channels, height, widht]` (такой порядок осей называется channels-first или NCHW).

Сверточные сети обычно собираются из последовательности слоев:

### Convolution
https://pytorch.org/docs/stable/nn.html#convolution-layers

По тензору бежит скользящее окно и в нем вычисляется свертка с ядром.
Обычно говорят о пространственных размерах сверток, например 1x1 или 3x3  свертки, подразумевая, что ядра имеют размер `[1,1,ch]` или `[3,3,ch]`.

Сейчас часто используются чуть более сложные варианты сверток: 
- dilated (atrous, дырявые), 
- depth-wise
- pointwise
- separable
- group


### Pooling
https://pytorch.org/docs/stable/nn.html#pooling-layers

Действуют аналогично свертках, но не имеют весов, а в бегущем окне вычисляется какая-нибудь функция, например max или mean.


### Global pooling (Adaptive Pooling)
https://pytorch.org/docs/stable/nn.html#adaptivemaxpool1d

Глобальные пулинги (в pytorch адаптивные) убирают пространственные размерности, превращая `[bs, ch, h, w]` в `[bs, ch, 1, 1]`.



Удобно выделять в сверточных сетях две части: полносверточную (body, feature extractor, тушка) и классификатор (head, голова).

Классификатор обычно состоит из полносвязных слоев (и где-то может обозначаться как MLP, MLP-head), и требует фиксированного размера тензоров (batch_size может варьироваться, но остальные размерности фиксированы).

Полносверточная часть обычно может работать на входах произвольных размеров (не меньше минимального).


Чтобы объединить эти две части используют какую-нибудь из операций: **Flatten** или **Global Pooling**.

#### Задание 1 (0.2 балла)

Реализуйте сверточную сеть, *2x(Conv+ReLU+MaxPooling) + Flatten + Dense*.

Ошибка классификации после обучения должна быть ниже 1.5%

Количество каналов и размеры фильтров выбирайте по желанию, начните с небольших чисел ~8-16.

**Hint: Для последовательности слоев без skip-connections удобно пользоваться оберткой `nn.Sequential`.**

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        <your code here>
        
    def forward(self, x):
        <your code here>
        
model = ConvNet()
# В качестве быстрой проверки корректности попробуем прогнать через сеть тензор нужного размера
# [bs, ch, h, w]
x = torch.zeros([4, 1, 28, 28])
model(x)

Стандартный цикл тренировки модели выглядит так:
```python
for epoch in epochs:
   model.train()
   for x, y in train_loader:
        ...
   model.eval()
   for x, y in val_loader:
        ...
```

В этом семинаре логгирование мы пишем самостоятельно, в дальнейшем вы можете использовать
этот код, tensorboardx или visdom на свое усмотрение.

Тренировочный цикл можно заменить на pytorch.ignite, но для учебных целей мы пока будем писать целиком так.

In [ ]:
def plot_history(log, name=None):
    """log is list of dictionaries like 
        [
            {'train_step': 0, 'train_loss': 10.0, 'train_acc': 0.0}, 
            ...
            {'train_step': 100, 'val_loss': 0.1, 'val_acc': 0.9},
            ...
        ]
    """
    if name is None:
        name='loss'
    train_points, val_points = [], []
    train_key = 'train_{}'.format(name)
    val_key = 'val_{}'.format(name)

    for entry in log:
        if train_key in entry:
            train_points.append((entry['train_step'], entry[train_key]))
        if val_key in entry:
            val_points.append((entry['train_step'], entry[val_key]))
    
    plt.figure()
    plt.title(name)
    x, y = list(zip(*train_points))
    plt.plot(x, y, label='train', zorder=1)
    x, y = list(zip(*val_points))
    plt.scatter(x, y, label='val', zorder=2, marker='+', s=180, c='orange')
    
    plt.legend(loc='best')
    plt.grid()
    plt.show()

Допишите тренировочный цикл:

In [ ]:
def train_model(model, optimizer, train_dataset, val_dataset, batch_size=32, epochs=10):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    log = []
    train_step = 0
    for epoch in range(epochs):
        model.train()
        for x, y in tqdm(train_loader):
            <your code here>
            
            loss = ....
            acc = ...
            
            log.append(dict(
                train_loss=loss,
                train_acc=acc,
                train_step=train_step,
            ))
            train_step += 1

        # валидационные метрики надо усредних за все валидационные батчи
        # hint: для аккумулирования величин удобно взять defaultdict
        tmp = defaultdict(list)
        model.eval()
        for x, y in tqdm(val_loader):
            with torch.no_grad():
                <your code here>
                
                acc = ...
                loss = ...
                
                tmp['acc'].append(acc)
                tmp['loss'].append(loss)
                
                
        log.append(dict(
            val_loss = np.mean(tmp['loss']),
            val_acc = np.mean(tmp['acc']),
            train_step=train_step,
        ))
        
        clear_output()
        plot_history(log, name='loss')
        plot_history(log, name='acc')

In [ ]:
# Обратите внимание на способ обхода весов модели:
def count_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum([np.prod(p.size()) for p in model_parameters])

model = ConvNet()
print("Total number of trainable parameters:", count_parameters(model))

opt = torch.optim.SGD(model.parameters(), lr=1e-3)
train_model(model, opt, mnist_train, mnist_val)

# Затухающие градиенты

Продолжаем экспериментировать с MNIST. 

В этом разделе нас будут интересовать особенности обучения глубоких сетей.

Эксперименты со сверточными сетями оказываются вычислительно дорогими, поэтому градиенты будем исследовать в полносвязных сетях.

**Hint: вам может пригодиться `model.named_parameters()` чтобы обойти слои модели**

**Задание 3 (0.2)** Реализуйте построение сети с произвольным числом (>1) полносвязных слоев с задаваемой функцией активации

In [ ]:
class DenseNet(nn.Module):
    def __init__(self, num_layers, hidden_size, activation):
        super().__init__()
        <your code here>
        
    def forward(self, x):
        <your code here>


model = DenseNet(10, 16, nn.Sigmoid)
# В качестве быстрой проверки корректности попробуем прогнать через сеть тензор нужного размера
# [bs, ch, h, w]
x = torch.zeros([4, 1, 28, 28])
model(x)

Добавьте в тренировочный цикл подсчет и отображение градиентов по слоям. Для отображения шумных величин можно воспользоваться оконным сглаживанием.

Нас интересуют нормы градиентов на каждом слое на каждом тренировочном шаге. 

Веса (weights) и смещения (biases) считать за разные величины.


**Hint: вам может пригодиться `model.named_parameters()`**

In [ ]:
def plot_grads(grad_log):
    """grad_log is list of dictionaries like 
        [
            {'train_step': 0, 'grad_layer.0.weight': 0.1, 'grad_layer.0.bias': 0.01, ...}, 
            ...
        ]
    """
    buffers = defaultdict(list)
    for entry in grad_log:
        for k, v in entry.items():
            buffers[k].append(v)
    
    names_to_plot = sorted(set(buffers.keys()).difference({'train_step'}))
    steps = buffers['train_step']
    
    plt.figure()
    plt.title('grads')
    
    for i, name in enumerate(names_to_plot):
        plt.semilogy(
            buffers[name], label=name, 
            color=plt.cm.coolwarm(i / len(names_to_plot)),
        )    
    
    plt.legend(loc='best')
    plt.grid()
    plt.show()


def train_model(model, optimizer, train_dataset, val_dataset, batch_size=32, epochs=10):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    grad_log = []
    log = []
    train_step = 0
    for epoch in range(epochs):
        model.train()
        for x, y in tqdm(train_loader):
            <your code here>
            
            # кроме обычных тренировочных действий, обойдите все веса сети и посчитайте нормы градиентов для них
            # добавьте в grad_log словарь вида {train_step: 0, grad_w0: 0.01, grad_w1: ...}.
            entry = {...}
            loss = ...
            acc = ...
            
            entry['train_step'] = train_step
            grad_log.append(entry)
            log.append(dict(
                train_loss=loss,
                train_acc=acc,
                train_step=train_step,
            ))
            train_step += 1

        tmp = defaultdict(list)
        model.eval()
        for x, y in tqdm(val_loader):
            with torch.no_grad():
                # здесь градиенты считать не обязательно
                <your code here>
                acc = ...
                loss = ...
                
                tmp['acc'].append(acc)
                tmp['loss'].append(loss)
                # <end>
                
        log.append(dict(
            val_loss = np.mean(tmp['loss']),
            val_acc = np.mean(tmp['acc']),
            train_step=train_step,
        ))
        
        clear_output()
        plot_history(log, name='loss')
        plot_history(log, name='acc')
        plot_grads(grad_log)

In [ ]:
# Проведите эксперимент с сигмоидой в качестве активационной функции
model = DenseNet(20, 10, nn.Sigmoid)
opt = torch.optim.SGD(model.parameters(), lr=1e-3)
x = train_model(model, opt, mnist_train, mnist_val)

In [ ]:
# Проведите эксперимент с relu в качестве активационной функции
model = DenseNet(20, 10, nn.ReLU)
opt = torch.optim.SGD(model.parameters(), lr=1e-3)
x = train_model(model, opt, mnist_train, mnist_val)

**Задание 4 (0.2 балла)**. Соберите ResNet с полносвязными слоями и проведите эксперимент с сигмоидой.


In [ ]:
class DenseResNet(nn.Module):
    def __init__(self, num_layers, hidden_size, activation):
        super().__init__()
        <your code here>

    def forward(self, x):
        <your code>

model = DenseResNet(10, 16, nn.Sigmoid)
# В качестве быстрой проверки корректности попробуем прогнать через сеть тензор нужного размера
# [bs, ch, h, w]
x = torch.zeros([4, 1, 28, 28])
model(x)

In [ ]:
model = DenseResNet(20, 10, nn.Sigmoid)
opt = torch.optim.SGD(model.parameters(), lr=1e-3)
x = train_model(model, opt, mnist_train, mnist_val)

**Задание 5 (0.1 балла)** 
Что вы можете сказать про градиенты на разных слоях по этим трем экспериментам?
(DenseNet[Sigmoid], DenseNet[ReLU], DenseResNet)?